In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [4]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [146]:
# Example 1
x = [2.0, 3.0, -1.0] # inputs
n = MLP(3, [4, 4, 1]) # MLP(inputs, layers=(4 node layer, 4 node layer, 1 node layer - output))
n(x) # passing the inputs to the Multi Layer Perceptron

Value(data=-0.3385994072604859)

In [147]:
# Example 2
xs = [
  [2.0, 3.0, -1.0], # expected result = 1.0
  [3.0, -1.0, 0.5], # expected result = -1.0
  [0.5, 1.0, 1.0], # expected result = -1.0
  [1.0, 1.0, -1.0], # expected result = 1.0
] # list of groups of inputs
ys = [1.0, -1.0, -1.0, 1.0] # desired targets for each group of inputs

ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
ypred

[Value(data=-0.3385994072604859),
 Value(data=-0.504481117999009),
 Value(data=-0.49052826352576057),
 Value(data=-0.37375254461563556)]

In [148]:
# Calculating Loss value for example 2 - using square function for loss
list_of_losses = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
# zip ex: a = [1,2,3] b = [a,b,c] zip(a,b) = [[1,a],[2,b],[3,c]]
# ygt means Y ground truth which is the expected result

list_of_losses
# returned a list of losses for each group of inputs

loss = sum(list_of_losses)
# total loss = the sum of all losses of expected results

loss

Value(data=4.184144839641647)

In [149]:
# calculating grade of nodes with derivative 

# because ypred is a list of output nodes of the MLP and we apply - and ** to those nodes which are instances of Value that has a -,** method
# it means that the result of list of losses and loss itself are Value objects

# losses = loss of ys0 + loss of ys1 + loss of ys2 + loss of ys3
# loss of ys0 = object MLP + loss function -> loss function = (ys0 - ygt0)**2 -> object MLP is a graph with the output node ys0

# doing backward() on loss will spread the gradient (derivate) to the loss function but also to the MLP
# due to that we have, 1) how much the nodes affect the loss function, but also, 2) how much the nodes affect the output node ys0

loss.backward() # always do one calculation per MLP creation because derivates add up and can acumulate
n.layers[0].neurons[0].w[0].grad
# how much this affects the loss or the output results
# first layer, first neuron, first weight (one weight per input on a neuron, each neuron has 3 inputs), gradient derivative

# the goal is to get a prediction or results as close as the ygt y ground truth value (expected result) to reduce loss as close to zero we can
# if we make a Weight with a possitive gradient higher we will get a higher output
# if we make a Weight with a negative gradient lower we will get a lower output
# if the output is closer to the expected result we will get a lower loss
# if the output is farther from the expected result we will get a higher loss

0.24024189042170316

In [150]:
# getting all the parameters of the MLP - values we can interact with to change the output (weights and biases)

n.parameters()
# node1w1,node1w2,node1w3,node1bias,node2w1,node2w2,node2w3,node2bias, ... 
# has all the objects inside we can modify them

[Value(data=-0.5230288503263478),
 Value(data=0.10290939418529921),
 Value(data=0.6149691881406345),
 Value(data=-0.5650960656513733),
 Value(data=0.0055766926217823976),
 Value(data=0.9987392215334487),
 Value(data=-0.3846799299051995),
 Value(data=-0.2260773625343271),
 Value(data=-0.6476436149471521),
 Value(data=-0.2139538772240397),
 Value(data=-0.5427440826293402),
 Value(data=-0.36273715859179645),
 Value(data=-0.018570376978292558),
 Value(data=-0.31023969845546606),
 Value(data=0.8746668788508833),
 Value(data=-0.8779532305726485),
 Value(data=-0.486705861136824),
 Value(data=0.4544363889284224),
 Value(data=0.27186030797596206),
 Value(data=-0.3560371383203891),
 Value(data=0.7894453494722551),
 Value(data=-0.2704499405374148),
 Value(data=0.5545171215117954),
 Value(data=-0.63688475044623),
 Value(data=0.6256998926971364),
 Value(data=-0.9061911247777024),
 Value(data=0.3423366933167886),
 Value(data=0.9729720044670649),
 Value(data=-0.10316811212932286),
 Value(data=0.69975

In [151]:
# Gradient decent

# we modify each parameter by a tiny STEP size in the direction of the gradient

# following the direction can get us closer to the loss
print({"g": n.layers[0].neurons[0].w[0].grad, "w": n.layers[0].neurons[0].w[0].data})
# 1) if grad is negative means that making W bigger the lost will be lower
# 2) if grad is possitive means that making W smaller the lost will be lower

# if the amount to add is (step * grad) means:
# 1) if grad is negative, W must get bigger -> (step(negative) * grad(negative)) -> step must be negative to cancel the grad
# 2) if grad is possitive, W must get smaller -> (step(negative) * grad(possitive)) -> step must be negative to take out the grad

# the previous is "following the direction of the gradient"

{'g': 0.24024189042170316, 'w': -0.5230288503263478}


In [152]:
# changing the weights
stepSign = -1

# update
for p in n.parameters():
    p.data += stepSign * 0.01 * p.grad
    
n.layers[0].neurons[0].w[0].data
# new weight

-0.5254312692305648

In [153]:
ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
print(ypred)
# Calculating Loss after changing weights
new_loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
print({"old loss": loss, "new loss": new_loss})
loss = new_loss
loss.backward() # backward pass -> recalculate gradient

[Value(data=-0.18278168566528033), Value(data=-0.437208144315173), Value(data=-0.41035056814230064), Value(data=-0.22528344305775064)]
{'old loss': Value(data=4.184144839641647), 'new loss': Value(data=3.564713157091937)}


In [154]:
# training the Model multiple times

iterations_training = 200 # iterations_training

print("Expected result:\n")
print(ys)
print("---------------------------------------\n")
for k in range(iterations_training):
    # Forward pass
    ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    # zero grad to flush previous gradients
    for p in n.parameters():
        p.grad = 0.0
    # Backward pass
    loss.backward()
    # Update
    for p in n.parameters():
        p.data += -0.01 * p.grad
    # Show status
    print(ypred)
    print(k, {"loss": loss.data})
    
# if the step is too large and goes over the point where loss is close to zero, the loss will change values into something that will not make sense
# to get closer to zero we could make the step size bigger with the risk of going over the 

Expected result:

[1.0, -1.0, -1.0, 1.0]
---------------------------------------

[Value(data=-0.18278168566528033), Value(data=-0.437208144315173), Value(data=-0.41035056814230064), Value(data=-0.22528344305775064)]
0 {'loss': 3.564713157091937}
[Value(data=-0.030499560862332614), Value(data=-0.3814262661487062), Value(data=-0.338935144060703), Value(data=-0.07755008437008608)]
1 {'loss': 3.042683737232215}
[Value(data=0.09860905341016384), Value(data=-0.3459805544412464), Value(data=-0.28702566980048577), Value(data=0.050181871423075106)]
2 {'loss': 2.6507339466599182}
[Value(data=0.19864568430294613), Value(data=-0.33192516572058717), Value(data=-0.25763651879685406), Value(data=0.15082125238572305)]
3 {'loss': 2.3607008071075635}
[Value(data=0.27428832966973543), Value(data=-0.33465767430840987), Value(data=-0.24742676944705866), Value(data=0.22776579004884379)]
4 {'loss': 2.132049981174014}
[Value(data=0.33265443146503304), Value(data=-0.3484709703248173), Value(data=-0.2512336338

[Value(data=0.9165498587103741), Value(data=-0.8841684515861655), Value(data=-0.8970633231386221), Value(data=0.8895542035608547)]
124 {'loss': 0.043175107083545855}
[Value(data=0.9169791766171166), Value(data=-0.8847198727414587), Value(data=-0.8976432440843009), Value(data=0.8901010542501633)]
125 {'loss': 0.04273664861442897}
[Value(data=0.9174024014719697), Value(data=-0.8852640362391384), Value(data=-0.8982147134052215), Value(data=0.8906405114666144)]
126 {'loss': 0.042306446962196306}
[Value(data=0.9178196710372956), Value(data=-0.8858010960082359), Value(data=-0.8987779244170313), Value(data=0.8911727349785826)]
127 {'loss': 0.04188427833870452}
[Value(data=0.9182311188526281), Value(data=-0.8863312015090662), Value(data=-0.8993330644759362), Value(data=0.8916978798557693)]
128 {'loss': 0.04146992681000695}
[Value(data=0.9186368743968636), Value(data=-0.8868544978970919), Value(data=-0.8998803152082252), Value(data=0.8922160966429454)]
129 {'loss': 0.04106318395971815}
[Value(d